In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import tensorflow as tf 
import matplotlib.pyplot as plt
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from PIL import Image, ImageFilter # dealing with images 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
        
directory = '/kaggle/input/early-detection-of-3d-printing-issues'
df_train = pd.read_csv(directory + '/train.csv')
df_test = pd.read_csv(directory + '/test.csv')

iteration = 1 

while iteration < 6: 

    df_train_0 = df_train[df_train['has_under_extrusion'] == 0].sample(25)
    df_train_1 = df_train[df_train['has_under_extrusion'] == 1].sample(25)

    img_path = '/kaggle/working/training_images/' + str(iteration) + '/0'
    os.makedirs(img_path)

    for index in range(df_train_0['img_path'].count()):
        image = Image.open(directory + '/images/' + df_train_0['img_path'].iloc[index])
        image = image.save(img_path + "/" + str(index + 1) + '.jpg')

    img_path = '/kaggle/working/training_images/' + str(iteration) + '/1'
    os.makedirs(img_path)

    for index in range(df_train_1['img_path'].count()):
        image = Image.open(directory + '/images/' + df_train_1['img_path'].iloc[index])
        image = image.save(img_path + "/" + str(index + 1) + '.jpg')

    train = ImageDataGenerator(rescale=1/255)
    
    training_data = train.flow_from_directory("/kaggle/working/training_images/" + str(iteration),
                                         target_size = (255,255),
                                         batch_size = 5,
                                         class_mode = 'binary')
    
    model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (5,5), activation = "relu", input_shape = (255,255,3)),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation = "relu"),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation = "relu"),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation = "relu"),
    tf.keras.layers.Dense(1, activation = "sigmoid")
])
    
    model.compile(
    loss = 'binary_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)
    
    model.fit(
    training_data,
    steps_per_epoch = 5,
    epochs = 10
)   
    
    model.save('/kaggle/working/model_test1_' + str(iteration) + '.h5')
               
    hue_list = []

    for index in range(df_test['img_path'].count()):
        image = Image.open(directory + '/images/' + df_test['img_path'].iloc[index])
        image = image.resize((255,255))
        img_arr = np.asarray(image)
        img_arr = np.expand_dims(img_arr, axis = 0)
        value = model.predict(img_arr)
        hue_list.append(value[0][0])
    
    hue_list1 = [round(x) for x in hue_list]

    df_test['has_under_extrusion'] = hue_list1
    df_final = df_test[['img_path','has_under_extrusion']]
    df_final.to_csv('/kaggle/working/test1_' + str(iteration) + '.csv', index = False)
               
    iteration += 1

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
'''
training_data = train.flow_from_directory("/kaggle/working/training_images",
                                         target_size = (255,255),
                                         batch_size = 5,
                                         class_mode = 'binary')
'''

In [ ]:
# training_data.class_indices

In [ ]:

'''
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (5,5), activation = "relu", input_shape = (255,255,3)),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation = "relu"),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation = "relu"),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation = "relu"),
    tf.keras.layers.Dense(1, activation = "sigmoid")
])
'''

In [ ]:
'''
model.compile(
    loss = 'binary_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)
'''

In [ ]:
'''
model.fit(
    training_data,
    steps_per_epoch = 5,
    epochs = 10
)
'''

In [ ]:
# model.save('/kaggle/working/model_test1.h5')

In [ ]:
'''
hue_list = []

for index in range(df_test['img_path'].count()):
    image = Image.open(directory + '/images/' + df_test['img_path'].iloc[index])
    image = image.resize((255,255))
    img_arr = np.asarray(image)
    img_arr = np.expand_dims(img_arr, axis = 0)
    value = model.predict(img_arr)
    hue_list.append(value[0][0])
'''

In [ ]:
# hue_list1 = [round(x) for x in hue_list]

In [ ]:
'''
df_test['has_under_extrusion'] = hue_list1
df_final = df_test[['img_path','has_under_extrusion']]
df_final.to_csv('/kaggle/working/test1.csv', index = False)
'''